In [71]:
pip install snorkel


[notice] A new release of pip available: 22.3 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [72]:
import pandas as pd
import random

In [73]:
tweet = pd.read_csv('TWEETSS.csv')

In [74]:
tweet

,Unnamed: 0,Date,User,Tweet
0,0,2022-12-31 23:11:28+00:00,BevonKimutai,@DanOjowa When Uhuru Kenyatta appointed Moody ...
1,1,2022-12-31 21:59:22+00:00,Dacha37533500,@edutheguru @PolycarpHinga Kwenda huko. Reject...
2,2,2022-12-31 21:47:40+00:00,kerubobree1,@Nesh_maina I miss Uhuru Kenyatta
3,3,2022-12-31 21:27:19+00:00,Lewisto90476962,@citizentvkenya This too much I think this reg...
4,4,2022-12-31 21:26:04+00:00,SaSamnjuguna16,@OleItumbi @WilliamsRuto @OleItumbi just to le...
...,...,...,...,...
4995,4995,2022-12-02 02:18:53+00:00,RGakwerere,"President Uhuru Kenyatta further noted, “We do..."
4996,4996,2022-12-02 02:18:52+00:00,RGakwerere,President Uhuru Kenyatta: “We have no problem ...
4997,4997,2022-12-01 22:21:57+00:00,gchahal,JUST IN: #BNNDRC Reports. \n\nThe work related...
4998,4998,2022-12-01 22:18:17+00:00,FatakiHugo,"@afrikarabia Bravo Uhuru Kenyatta, Kagame doit..."


In [75]:
#Train and Test
from sklearn.model_selection import train_test_split
tweet_train,tweet_test = train_test_split(tweet,train_size=0.6,random_state=2)

In [121]:
print(tweet_train.shape)

(3000, 5)


In [122]:
print(tweet_test.shape)

(2000, 4)


In [76]:
print(tweet.shape)

(5000, 4)


In [77]:
tweet_train[["Date","User","Tweet"]].sample(20, random_state=2)

,Date,User,Tweet
2572,2022-12-17 05:53:00+00:00,RhingoMatulu,@IlungaJpaul @UhuruKenyatta @fatshi13 @Lukonde...
3274,2022-12-12 04:12:23+00:00,AtamboJanai,Is it grounds or stadium??#JamhuriDay2022 #Rai...
1020,2022-12-24 13:32:24+00:00,Orawebhostkenya,♦ May Christmas bring a joyful end to the year...
3176,2022-12-12 16:10:31+00:00,StindoP,@MrRightke like uhuru Kenyatta
3115,2022-12-13 04:53:12+00:00,trapshoe254,"Truth be told, Former President, Uhuru Kenyatt..."
3107,2022-12-13 05:36:10+00:00,andivamorgan,"Uhuru Kenyatta played Raila Odinga, i hate to ..."
4174,2022-12-06 16:42:00+00:00,midudamozez,@KTNNewsKE They feel that ALL the roads were d...
1688,2022-12-23 06:11:27+00:00,PeculiarDr,😆😆✌️\n#Uhurukenyatta\n#Chebukati https://t.co/...
1556,2022-12-23 08:00:54+00:00,ballet_jaz,Kids ballet costume!! Tutu (Age 3-11yrs) &amp;...
97,2022-12-31 04:25:55+00:00,MtiraRophus,Consumers will be hit by an electricity price ...


In [78]:
print(tweet_train.shape)

(3000, 4)


In [79]:
#Define our labeling function
from snorkel.labeling import labeling_function,PandasLFApplier,LFAnalysis

In [80]:
#Constants for our labels
Irrelevant = 1
Relevant = 0
Unknown = -1

In [138]:
###Keyword_Lookup
@labeling_function()
def lf_keyword_lookup(x):
    keywords = ["whatsapp","size","retweet","like","order","call","ksh","kshs","price","offer","affordable","shop","prices"]
    return Irrelevant if any(word in x.Tweet.lower() for word in keywords)else Unknown

In [139]:
###Pattern Lookup/Regex Fxn
import re
@labeling_function()
def lf_regex_contains_call(x):
    return Irrelevant if re.search(r"call.*whatsapp",x.Tweet,flags=re.I)else Unknown

In [149]:
###Pattern Lookup/Regex Fxn
import re
@labeling_function()
def lf_regex_contains_price(x):
    return Irrelevant if re.search(r"price.*call",x.Tweet,flags=re.I)else Unknown

In [150]:
###Apply Fxn
lfs = [lf_keyword_lookup,lf_regex_contains_call,lf_regex_contains_price]
applier=PandasLFApplier(lfs=lfs)
L_train = applier.apply(tweet_train)

100%|██████████| 3000/3000 [00:00<00:00, 4793.68it/s]


In [151]:
#Label Matrix
L_train

array([[-1, -1, -1],
       [-1, -1, -1],
       [-1, -1, -1],
       ...,
       [ 1, -1, -1],
       [-1, -1, -1],
       [ 1, -1, -1]])

In [154]:
###Evaluate the Performance
LFAnalysis(L=L_train,lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
lf_keyword_lookup,0,[1],0.109000,0.007000,0.0
lf_regex_contains_call,1,[1],0.006667,0.006667,0.0
lf_regex_contains_price,2,[1],0.000333,0.000333,0.0


In [153]:
###LabelModel
from snorkel.labeling.model import MajorityLabelVoter,LabelModel

In [156]:
#Build the model
label_model = LabelModel(cardinality=2,verbose=True)
label_model.fit(L_train=L_train,n_epochs=500,log_freq=100,seed=123)

INFO:root:Computing O...
INFO:root:Estimating \mu...
100%|██████████| 500/500 [00:01<00:00, 459.51epoch/s]
INFO:root:Finished Training


In [158]:
###Unlabeled Dataset
tweet.head()

,Unnamed: 0,Date,User,Tweet
0,0,2022-12-31 23:11:28+00:00,BevonKimutai,@DanOjowa When Uhuru Kenyatta appointed Moody ...
1,1,2022-12-31 21:59:22+00:00,Dacha37533500,@edutheguru @PolycarpHinga Kwenda huko. Reject...
2,2,2022-12-31 21:47:40+00:00,kerubobree1,@Nesh_maina I miss Uhuru Kenyatta
3,3,2022-12-31 21:27:19+00:00,Lewisto90476962,@citizentvkenya This too much I think this reg...
4,4,2022-12-31 21:26:04+00:00,SaSamnjuguna16,@OleItumbi @WilliamsRuto @OleItumbi just to le...


In [162]:
#apply our label model on our unlabeled dataset
tweet_train['Intention'] = label_model.predict(L=L_train,tie_break_policy="abstain")

In [163]:
tweet_train

,Unnamed: 0,Date,User,Tweet,Intention
0,2805,2022-12-15 07:56:12+00:00,jumaf3,There are some bad policies that Uhuru Kenyatt...,-1
1,1563,2022-12-23 07:53:40+00:00,BenjaminkamauK,Truth be told Jaden was a vibe #UhuruKenyatta,-1
2,315,2022-12-29 12:05:17+00:00,Chepchos,@RobertAlai You are ins@ne yourself. All these...,-1
3,828,2022-12-25 02:25:14+00:00,ScerahChui,@paulinenjoroge Long live President Uhuru Keny...,-1
4,23,2022-12-31 17:12:43+00:00,georgepinto0000,Itumbi reveals finer details of his grand fall...,-1
...,...,...,...,...,...
2995,3335,2022-12-11 16:26:30+00:00,Wyclifffe_KE,HOT DEAL🔥Order these Amazing Shoes at BEST PRI...,1
2996,1099,2022-12-24 09:30:23+00:00,sahsama7,@Atito4Atito @rigathi @WilliamsRuto Uhuru keny...,-1
2997,2514,2022-12-18 05:32:43+00:00,Cjamehk,President Uhuru Kenyatta 01/02/2020:He instruc...,1
2998,3606,2022-12-11 06:21:31+00:00,lilechikenya,@the_Betraya @KBonimtetezi My friend the resur...,-1


In [167]:
tweet_train['Intention'].value_counts()


-1    2673
 1     327
Name: Intention, dtype: int64

In [168]:
tweet_test

,Unnamed: 0,Date,User,Tweet
3566,3566,2022-12-11 07:53:13+00:00,dogutha,MURATHE: Uhuru Kenyatta has to resign as Azimi...
4252,4252,2022-12-06 13:16:55+00:00,bosolo_m,@DeniseIkali @wembi_steve Oza ndoki! \nPour to...
1918,1918,2022-12-22 21:40:02+00:00,ViralCiti,"How to Unlock M-Kopa Phones Use Samsung, Nokia..."
4111,4111,2022-12-07 04:28:31+00:00,Denoden74513302,Plzzz my Uhuru kenyatta our former president s...
1471,1471,2022-12-23 09:33:01+00:00,__Willie_,Get Festive season discounts for cleaning serv...
...,...,...,...,...
4486,4486,2022-12-05 19:45:19+00:00,Githunguri85,@SakajaJohnson @UKinKenya And uhuru kenyatta 😮‍💨
2188,2188,2022-12-22 11:53:56+00:00,Catherine4785,@K24Tv Uhuru Kenyatta...
2141,2141,2022-12-22 13:56:38+00:00,RICHARD81691342,@K24Tv Uhuru Kenyatta
2012,2012,2022-12-22 19:10:01+00:00,kanam_selfhelp,@K24Tv Uhuru Kenyatta
